# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770894353546                   -0.53    9.0
  2   -2.772144162541       -2.90       -1.32    1.0
  3   -2.772170302834       -4.58       -2.53    1.0
  4   -2.772170678958       -6.42       -3.44    1.0
  5   -2.772170722890       -7.36       -4.34    2.0
  6   -2.772170723001       -9.95       -4.91    1.0
  7   -2.772170723014      -10.89       -5.84    1.0
  8   -2.772170723015      -12.47       -5.71    2.0
  9   -2.772170723015      -12.44       -6.34    1.0
 10   -2.772170723015      -13.75       -7.46    1.0
 11   -2.772170723015      -13.91       -8.04    2.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770770665213                   -0.53    9.0
  2   -2.772051727839       -2.89       -1.31    1.0
  3   -2.772082786885       -4.51       -2.58    1.0
  4   -2.772083411788       -6.20       -3.75 

1.7735581911778093

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770817698641                   -0.52    9.0
  2   -2.772062440799       -2.90       -1.32    1.0
  3   -2.772083157059       -4.68       -2.48    1.0
  4   -2.772083351284       -6.71       -3.20    1.0
  5   -2.772083416161       -7.19       -3.90    2.0
  6   -2.772083417578       -8.85       -4.42    1.0
  7   -2.772083417810       -9.63       -5.41    2.0
  8   -2.772083417810      -12.47       -6.12    1.0
  9   -2.772083417811      -12.82       -7.24    2.0
 10   -2.772083417811   +  -14.10       -7.35    1.0
 11   -2.772083417811      -14.07       -8.41    1.0

Polarizability via ForwardDiff:       1.7725349670616075
Polarizability via finite difference: 1.7735581911778093
